In [1]:
# -*- coding: utf-8 -*-
"""
@author: JulienWuthrich
"""
import os
%matplotlib inline
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf


class DeepLabModel(object):
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()
        graph_def = self.read_tarfile(tarball_path)
        if graph_def is None:
            raise RuntimeError('Cannot find inference graph in tar archive.')
        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
        self.sess = tf.Session(graph=self.graph)

    def read_tarfile(self, tarball_path):
        """Read the tarfile"""
        graph_def = None
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        return graph_def

    def run(self, image):
        """Runs inference on a single image.

            Args:
            -----
                image: A PIL.Image object, raw input image.

            Returns:
            --------
                resized_image: RGB image resized from original input image.
                seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        batch_seg_map = self.sess.run(self.OUTPUT_TENSOR_NAME, feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]

        return resized_image, seg_map


def unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model


def create_pascal_label_colormap():
    """Creates a label colormap used in PASCAL VOC segmentation benchmark.

    Returns:
    A Colormap for visualizing segmentation results.
    """
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return colormap


def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()
    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    LABEL_NAMES = np.asarray([
        'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
        'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
        'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
    ])
    FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
    FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()


def sav_segmentation(image, seg_map):
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    #plt.imshow(image)
    #plt.imshow(seg_image, alpha=0.7)
    #plt.savefig("toto.png")


def run_viz(img):
    orignal_im = Image.open(img)
    resized_im, seg_map = MODEL.run(orignal_im)

    # vis_segmentation(resized_im, seg_map)
    sav_segmentation(resized_im, seg_map)

    return resized_im, seg_map
    
def run_visualization(url):
    """Inferences DeepLab model and visualizes result."""
    try:
        f = urllib.request.urlopen(url)
        jpeg_str = f.read()
        orignal_im = Image.open(BytesIO(jpeg_str))
    except IOError:
        print('Cannot retrieve image. Please check url: ' + url)
        return

    #  print('running deeplab on image %s...' % url)
    resized_im, seg_map = MODEL.run(orignal_im)

    # vis_segmentation(resized_im, seg_map)
    sav_segmentation(resized_im, seg_map)
    

In [2]:
MODEL_NAME = None
models = ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']
_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug': 'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval': 'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug': 'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval': 'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
for MODEL_NAME in models:
    break

_TARBALL_NAME = 'deeplab_model.tar.gz'
model_dir = tempfile.mkdtemp()
tf.gfile.MakeDirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME], download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

# Here return the image saved as toto.png

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


In [3]:
import glob

files = glob.glob("../data/images/renault/*")

In [4]:
import scipy.misc
import multiprocessing as mp

In [5]:
def toto(file):
    print(file)
    resized_im, seg_map = run_viz(file)
    res = np.array(resized_im)
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            for k in range(res.shape[2]):
                if seg_map[i][j] > 0:
                    res[i][j][k] = res[i][j][k]
                else:
                    res[i][j][k] = 255
    file = file.replace("images", "images_clean")
    scipy.misc.imsave(file, res)
    
    return file

In [6]:
for i in files:
    toto(i)

../data/images/renault/renault_laguna_89.jpg
../data/images/renault/renault_clio_gt_39.jpg
../data/images/renault/renault_pulse_34.jpg
../data/images/renault/renault_captur_gris_81.jpg
../data/images/renault/renault_twingo1_18.jpg
../data/images/renault/renault_espace_gris_29.jpg
../data/images/renault/renault_laguna_3.jpg
../data/images/renault/renault_espace_noir_86.jpg
../data/images/renault/renault_zoe_gris_6.jpg
../data/images/renault/renault_pulse_20.jpg
../data/images/renault/renault_rs16_69.jpg
../data/images/renault/renault_kadjar_bleu_3.jpg
../data/images/renault/renault_logan_gris_40.jpg
../data/images/renault/renault_talisman_gris_36.jpg
../data/images/renault/renault_twingo1_24.jpg
../data/images/renault/renault_captur_rouge_31.jpg
../data/images/renault/renault_talisman_gris_22.jpg
../data/images/renault/renault_captur_orange_15.jpg
../data/images/renault/renault_twingo1_30.jpg
../data/images/renault/renault_espace_gris_15.jpg
../data/images/renault/renault_clio_gt_11.jpg

../data/images/renault/renault_pulse_blanc_89.jpg
../data/images/renault/renault_twingo_gt_38.jpg
../data/images/renault/renault_laguna_noir_87.jpg
../data/images/renault/renault_twingo_gt_10.jpg
../data/images/renault/renault_f1_83.jpg
../data/images/renault/renault_espace_bleu_70.jpg
../data/images/renault/renault_laguna_gris_28.jpg
../data/images/renault/renault_megane_gt_55.jpg
../data/images/renault/renault_kangoo_6.jpg
../data/images/renault/renault_espace_bleu_64.jpg
../data/images/renault/renault_scenic_34.jpg
../data/images/renault/renault_captur_bleu_33.jpg
../data/images/renault/renault_captur_bleu_27.jpg
../data/images/renault/renault_scenic_20.jpg
../data/images/renault/renault_koleos_blanc_64.jpg
../data/images/renault/renault_f1_54.jpg
../data/images/renault/renault_laguna_noir_50.jpg
../data/images/renault/renault_clio_31.jpg
../data/images/renault/renault_clio_rs_31.jpg
../data/images/renault/renault_kadjar_39.jpg
../data/images/renault/renault_kangoo_jaune_3.jpg
../da

../data/images/renault/renault_talisman_noir_65.jpg
../data/images/renault/renault_captur_gris_41.jpg
../data/images/renault/renault_fluence_73.jpg
../data/images/renault/renault_logan_gris_94.jpg
../data/images/renault/renault_megane_rs_9.jpg
../data/images/renault/renault_megane_jaune_56.jpg
../data/images/renault/renault_talisman_break_16.jpg
../data/images/renault/renault_pulse_jaune_17.jpg
../data/images/renault/renault_laguna_61.jpg
../data/images/renault/renault_captur_gris_69.jpg
../data/images/renault/renault_koleos_18.jpg
../data/images/renault/renault_rs16_81.jpg
../data/images/renault/renault_logan_gris_80.jpg
../data/images/renault/renault_twingo_rs_33.jpg
../data/images/renault/renault_rs01_72.jpg
../data/images/renault/renault_koleos_gris_43.jpg
../data/images/renault/renault_wind_12.jpg
../data/images/renault/renault_zoe_gris_20.jpg
../data/images/renault/renault_captur_jaune_30.jpg
../data/images/renault/renault_captur_jaune_24.jpg
../data/images/renault/renault_clio_t

../data/images/renault/renault_alaskan_50.jpg
../data/images/renault/renault_koleos_noir_12.jpg
../data/images/renault/renault_wind_gris_94.jpg
../data/images/renault/renault_wind_noir_13.jpg
../data/images/renault/renault_kangoo_blanc_67.jpg
../data/images/renault/renault_zoe_noir_59.jpg
../data/images/renault/renault_clio_gt_1.jpg
../data/images/renault/renault_twingo_rs_1.jpg
../data/images/renault/renault_clio_v6_68.jpg
../data/images/renault/renault_pulse_rouge_14.jpg


/Users/az02210/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


...

../data/images/renault/renault_megane_noir_46.jpg
../data/images/renault/renault_talisman_65.jpg
../data/images/renault/renault_clio_rouge_68.jpg
../data/images/renault/renault_laguna_blanc_77.jpg
../data/images/renault/renault_captur_59.jpg
../data/images/renault/renault_twingo3_4.jpg
../data/images/renault/renault_koleos_noir_80.jpg
../data/images/renault/renault_talisman_71.jpg
../data/images/renault/renault_laguna_blanc_63.jpg
../data/images/renault/renault_twingo_rs_63.jpg
../data/images/renault/renault_talisman_59.jpg
../data/images/renault/renault_clio_rouge_54.jpg
../data/images/renault/renault_talisman_blanc_57.jpg
../data/images/renault/renault_kadjar_gris_34.jpg
../data/images/renault/renault_megane_rs_26.jpg
../data/images/renault/renault_zoe_gris_70.jpg
../data/images/renault/renault_twingo2_32.jpg
../data/images/renault/renault_wind_42.jpg
../data/images/renault/renault_captur_65.jpg
../data/images/renault/renault_koleos_gris_13.jpg
../data/images/renault/renault_clio_tro

../data/images/renault/renault_zoe_400km_6.jpg
../data/images/renault/renault_megane_bleue_58.jpg
../data/images/renault/renault_alaskan_28.jpg
../data/images/renault/renault_clio_v6_38.jpg
../data/images/renault/renault_kangoo_19.jpg
../data/images/renault/renault_clio_rouge_82.jpg
../data/images/renault/renault_kadjar_noir_59.jpg
../data/images/renault/renault_wind_noir_57.jpg
../data/images/renault/renault_kangoo_blanc_23.jpg
../data/images/renault/renault_wind_80.jpg
../data/images/renault/renault_koleos_noir_56.jpg
../data/images/renault/renault_alaskan_14.jpg
../data/images/renault/renault_pulse_blanc_6.jpg
../data/images/renault/renault_zoe_noir_35.jpg
../data/images/renault/renault_kadjar_noir_71.jpg
../data/images/renault/renault_clio_v6_10.jpg
../data/images/renault/renault_kangoo_31.jpg
../data/images/renault/renault_pulse_rouge_78.jpg
../data/images/renault/renault_zoe_noir_21.jpg
../data/images/renault/renault_megane_bleue_70.jpg
../data/images/renault/renault_zoe_gris_65.

../data/images/renault/renault_twingo_62.jpg
../data/images/renault/renault_espace_gris_2.jpg
../data/images/renault/renault_rs17_34.jpg
../data/images/renault/renault_pulse_noir_6.jpg
../data/images/renault/renault_wing_sport_56.jpg
../data/images/renault/renault_rs17_20.jpg
../data/images/renault/renault_zoe_blanc_45.jpg
../data/images/renault/renault_espace_initial_40.jpg
../data/images/renault/renault_clio_noire_12.jpg
../data/images/renault/renault_espace_initial_68.jpg
../data/images/renault/renault_zoe_25.jpg
../data/images/renault/renault_scenic_noir_58.jpg
../data/images/renault/renault_zoe_8.jpg
../data/images/renault/renault_kangoo_jaune_33.jpg
../data/images/renault/renault_twingo2_2.jpg
../data/images/renault/renault_zoe_31.jpg
../data/images/renault/renault_clio_39.jpg
../data/images/renault/renault_fluence_rouge_8.jpg
../data/images/renault/renault_kadjar_31.jpg
../data/images/renault/renault_clio_rs_39.jpg
../data/images/renault/renault_zoe_blanc_86.jpg
../data/images/r

../data/images/renault/renault_rs17_4.jpg
../data/images/renault/renault_megane_noir_21.jpg
../data/images/renault/renault_scenic_jaune_11.jpg
../data/images/renault/renault_megane_noir_4.jpg
../data/images/renault/renault_wind_7.jpg
../data/images/renault/renault_megane_rs_2.jpg
../data/images/renault/renault_logan_40.jpg
../data/images/renault/renault_fluence_50.jpg
../data/images/renault/renault_talisman_noir_46.jpg
../data/images/renault/renault_koleos_13.jpg
../data/images/renault/renault_captur_gris_62.jpg
../data/images/renault/renault_captur_gris_89.jpg
../data/images/renault/renault_twingo1_10.jpg
../data/images/renault/renault_alaskan_blanc_3.jpg
../data/images/renault/renault_captur_orange_35.jpg
../data/images/renault/renault_espace_gris_35.jpg
../data/images/renault/renault_megane_trophy_81.jpg
../data/images/renault/renault_rs16_75.jpg
../data/images/renault/renault_laguna_81.jpg
../data/images/renault/renault_logan_gris_60.jpg
../data/images/renault/renault_rs16_61.jpg
.

OSError: cannot identify image file '../data/images/renault/train-images-idx3-ubyte.gz'

In [ ]:
# res = pool.map(toto, files)

In [ ]:
#imgs = [
#    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ8hGGx-JpaBEORmGDZqnYl5hREWMxiIZBaXin0GCy-CPi00OR-",
#    "http://www.lepoint.fr/images/2014/10/03/soiree-2851614-jpg_2488588_660x281.JPG",
#    "https://group.renault.com/wp-content/uploads/2016/11/thierry-koskas160x140.jpg",
#    "https://static.lexpress.fr/medias_11518/w_1802,h_1352,c_crop,x_0,y_0/w_640,h_358,c_fill,g_center/v1497427807/carlos-ghosn-patron-de-l-alliance-renault-nissan-le-7-mars-2017-a-geneve_5897531.jpg",
#    "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTExMWFhUXFxgYGBgXGB4dGBgZGhoXGhodHRsYHSggGB0lHhgYITEhJSkrLi4uGCAzODMtNygtLisBCgoKDg0OGhAQGy0lHyYtLS0vLS0tLS0tLy0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tK//AABEIAMIBAwMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAFBgQHAAIDAQj/xABLEAACAQIDBAcEBQkHAwIHAAABAhEAAwQSIQUGMUETIlFhcYGRBzKhsRRCwdHwI1JicoKSsuHxFiQzU6LC0hVDk1RzNGODhJTD0//EABoBAAMBAQEBAAAAAAAAAAAAAAABAgMEBQb/xAAvEQACAgECBAUDAgcAAAAAAAAAAQIRIQMSBDFBUQUTImGBcbHRkaEGFiMyM0JS/9oADAMBAAIRAxEAPwA4N28CvFLXmVrouzcAvOyP/qD7Gqt0vd4Hlx7qJWt6b6qFVkAH/wAq388k1zZNFQ8hcCPrWv3p+2sbF4EaTaP7BP8AtpIO92K5XmHgFHyFR7+8+KMZsRd4yOudDqNPjSyP0lik4cLmFsFe1bDEfBKHbN97aC8ulusPB0W4P4qVsZvzi3QWzeYAQOrCkgfqgGaYt1nL3MQSZ6S3h3k8SWwtoGe+Qacb6kTqsA28NKWNrW8rT202ta0oJtbCSDrVIgV7pri9yBNd8RbA4n4VHayGUgNyPKtEhnNccv4FWX7Ptzlxtg3WuhQGKxGvAGePfVf4LdvMgZsRbtk6hWVyY5HqKQJ9aKYKxiLKlbV5WB4hHj/S+Un0qJq+RUWkdd4cB0F+5azBsjESOBoBFTskSbhYGdcwg/GoIdS5AM/dTimlkTeR49m1sDFk9tpxxPap+yn3ZGhw/wCwPRopJ9niD6Qp7bbfKnLAuALR5Bz8LrD7KTEuYC3v3kxGEv3BbICtcI90HXKh5jvPpS3c3/xn+cR4Jb/4UQ9pCBnc5v8AvGDx+qKR2woicx9P50qKsZv7a4zQm88HhoBPoKtLdDbtjEYdZLG6FIYuZkiCY14dYVRagMFUuSF4DsnjT17PmVGCTDMLhAPEwLJ+2lKI0wzvSf7ttL9e1/BhqqnLVr7xMPo+P0+vb/gw9Vfbx1s+6RpVpEWQm4jxqTbtliAoJJ4ACSfIVr0qk5mPkDxHKTyGn441mI3jNtSlthbnkkDTvMy37RNUAR/6FiIno47mZVb9xmDfCoF2yy+8pHKCDM+FcMHjbjayzdgH8hXe+uIuwOhutEAEIx8tBRQHNDrwNOW6e897BAqgBDakGNY/rSla2PiyRGFxB8LNw/7aIYHYWNIgYS/I0/wn+0VMoWNOhm2jvBexV7pGgQIAHAD1qYhdx1zPiaG7M3axvPDXR4qR86YU2JiVWTaYDTjHPTtqNiQOTZvh8MVwlwq2X3zHFdEHLl5EVV+3WbpLNvKZGGwiADWf7vaI4DiZ4RVoYzEBMObZhswfNlMwIggEc+Xnz4UB3Z2WDeW9dKtcdR0fCQqhUXTmwRQO4CmmhpMRr+zriMVdcrDiCRp6VlH8XhGa5cLiWzuCe8MR9lZS3F7SB0E8x6GtHtQJ09K62LqqkvcVdefH7vjUa7tFCSM4KAgMV4wZ5fCaStiOfSd4+FGNlbvX8Uge1azqCVLB0Go1iGYEDrD76jWb9stBVQCsopAgAdsjjpP9KsP2XBRZvKI/xc2nLMBw7B1aptbqITFS37PdocehTxNxP+Rim3dXdTGWLoNxVCGxbDHpAYuIWUgAcisGe34PlphUnpaGytohjc3EnnaH7R/41FxO4eIP/csDxdv+FWI1yo1xqEJxKjv+yjEs5Y4nDgdgNw/7BWH2S3eWMt/+Nvvq1WNaE1qIrI+yq7lhsZb/APEx/wB1cE9lF2ZGPUf/AG7fbcE1agr2gRWNv2WXGkNtCBMj8gSOEcDe0rphvY7bUy2PZj+jh4+d01ZBFbrQArbtbiJhLvSfSblzQjKUAAkR2n8Cjg2HayBM9yAWM6fWdn7P0oogK9FKkAAxO5OFuAB2uwO9OwDmh7Kir7Mtnc1vHxun7AKbBXRaVIYpL7NdnDhZJ/Wu3f8Aa4ojg91MJZdXtWFVlDAHM597Lm95jxyj0o+K0ekMgXtn2XDq9m2y3IzhlkNAAEg8dFX0qJZ3bwSe7gsMPCwn/Gi0VrFWSR0wFke7YsjwtIPsqTZtqvuog8FA+Qr0CtlpMZ3W6e2ts5PM1yWuq1m2ykaNPOor26mNXC5QmFEZye00B3r6VsM4tyWOUCDH1hzJ7KPuKjYhRAntH8qqxCU+7xXDqrPluaSZ6q6mQo0mdBXfYez7YuW3zFujkBmOgESYA7Z56611xeKtvjjYAZnOh5IAEzeJ0oNtjHXw9y2sW7dtgi5V6znTTXiQOyKii77jTeuDMYiJPzr2gNrCBAFuElwOsS5Jk66mOUx5VlYuSTN1pNqxhwW6eEtCBdcjTi6cvBRWuL3W2fcZS7AlZyzcEiSrSO+UHx7aMf2Xs/mOfF2++vf7L2P8s+bH76uutHPfuLlzcTZbHMZJgjS+w0Oh91hRPYexMJhcwwsjNBabjPwmPfYxxPCiH9mcP/kr51n/AEy1Z1toqk6HLzrRNt5JpG927EAczB8KH43eixbum0xbMHt2yY0BuKzKZJ1HV1jtqXc1y+I+/wCytbmEtsSWtoSeJKgk6R8iRTjF27Ksi296rBZVBbrmFOU6kcQQRI5nwWiztQbEYOzh7DtbsI3RI9xVPElZeMzAkSRx1ilk7/3P/TKD/wC6T/8ArFdehwerrpvTV17oynqRjzY9E1rNV5jPaJdRGf6Mkjl0h1E8Zy6dtRB7TrvQm79HQdgzEzrHGBXZHwfjGrUPbmvyR50O5ZxblW4NU4fa7f5Ya1+8331yue1zF/Vs4ceIc/7xWy8A45/6/uvyHmxLnJrdapAe1nHf5eG/cf8A/rTR7PN+sTjcU1m8toKLTOMisDIZBxZjpDGp1vA+L0dOWrNKlzyC1It0WTNALG+2CdXcXWypba6SbVxc1tTDMmZB0gB/Nmjymqmw272KGFRTbvOz4HHWQjqB0DZwUVYAjpAPrydNCOFeQaFgJvdhvyWbpU6a6LVvpLL28zkAj31Ghka8JrzE76Ya3atXm6TJcvNZELqGRmR2YTogKmT2RSvd2FdxK4a30eLTI9ws+JdWNsthnRHtw7Qq3MhA7dYqDhd1toX7Nu1dVbB6HGm4XCupuYq8TlUJcGVsuoYyBm560UA+Y7e21aOIUo5OHu4e08RBOIyZCuuoGcTMcDxqLjN7YwlzF28O7pae8jrnVSosllZteIldANdaXP7PY650ouWlBxR2fcuP0ixZfDC2LoIBJcno5GWR1tSKmpsTG/RcZgzZtBLxxbJd6bX8szsgNsW9B1hJzHwpUhhra28T4fZ9zG3MPDIoboukBkFlA66qQNGnhUC7vhcRntXcOqXkxGFtMq3c6FMSYV1bIpJENoVHCp+9GxbmJ2dcwiFRce2iAsTlBUoTJAJjqnlQy9upfvO9+69pbr4jCXCqZii28KSQoZgCzMWYyQBr3UwO+zt5MRdfOtqz9HNzEWxNyLwFnMocqYDBnUjKuoEGaH2t48fdXZ7W2wqDGqfetXGKMttrjH/FAIOWAPjXXD7k3Fu2yb1o27N/EXbf5I9Llvi51GfNEKbhOg17oojgN1TbTZ69NJwQf6mlwtba3+d1Imec8O+kBEwO9d5sXiLbZOhy4gYcgdY3ML0a3ZJOozO0dy0xbp7Qe/gsNeuEF7llHaBAllBMDlS5gvZ/ZtiyRdfprfSl7us3emV1eVLFVnPOnYKMbt7FfCW1tHE3LyIioiutsBAugjIoJ006xNS0NB1rlcnatWatSako8Y0M21gFvIFZmAzT1DBMAmJqexrliJjTv+R7KAFvdzZttbxK2wOjzjPxdi2Qak8x1vWh162v0669x0ARxkVnUEEqpzQx4CfWlza+0bjXHVpXrSFkgrpwgGJ4TIqPdxoZV6RBcdRAZy05RwHVYTx50K0VKNliOqkyACDwIOhHI8ayh+HxYCKOEKug4DQVlcz5nUnSK+ffIn67n9+uTb2T+cf2WoNgNnveLhWKhfrZS8mJjq8NOdcFGXOHBlDrPdrp2T9ta+VA5nKVWHH3lB+q37h+0Uz+zzbIuYoIFYTbfUrA0ynj5Ul7E2DdxKm4GW2msTJJ+4d5NMXs4weTH5bjRct5tFIKujqwkc+P2VShCLwJ76tlvsdV8fsNbluda0se0fGNawLlBmZntoBMTLD14cK3Mg7jcTbe26C4hLo4ADCTII0E661Tex9o9MgJ94aN9h8/vqBZ2pjGAAuJ7wOTUDMOHPQihmzHa3eCwVzaQe/UT8Na+l/h3bNa0eqSa+iu/uc/FQdJjbet5lbqlhBBjs4H50sY64OjS2DOU66HWO2PGp+ItYhXIV2BABBXhB0OszGvM00YPG9DaW10hEqM35EMmYtNw5uMENGugrk/mLXivRFc/c2hwKvLK7vWCVDgDnOXlHMip2z8FnskZQxmV7ZGkHuj51GxF1AzqCSM2kcOc+WpAo+cEVw63US4LgykwBlZSCdADMAaTFe94v4xLhdKEdvqkr7Vj8/YiGh5km08IFCxcZCpW0QAxGVAHGWZyldWg8jxox7LccljFm7dbKnROuYgxJKkDTidOFStjXr19gFzHPAUMVglo0E9vfWzbEfD3QLiiwubRbiNDjL1isaNrpp6189oeJakPDtaE3ucmkreV3f2+TeWjCWqkuxc2CxaXUDowZTwIrw4hxl6hM8dCMumvjr61Xe5W01wnSB2DrdyMpXhm6wfVogAZdeOtPex9s2sRnycUIDAkHiJEFSQRxGh4g140ZWhThtdEbbe3jhwpdYzSAJ1mNdeELp4zpWbK2+18lUVcygMQGBGUkjQzx0GkDiJiaXfa5s83LFpwSMjMDBP1ssSB+rUD2TYPLddgxIyGZ7CViBPaB8aHLNDULjZYzXroV2KiQJAB4x72s6d01Szb64g3S1xTLEgMGPVM6FckER3T4cavG4sqR2gj4VQ5wUYhUZVDK+UxyYnXgdYNKborTimXFu9ir17DYe6WEkS/wClBYHgvhwip7WbhjrgHSTr2CQFiOMme+vdjbOFmylpJZUWJ7eJJ7tSalVZmyG9m6Y64Gp8YI05awfWa8uYQkCXOYTDQJ1IM/CKmVqTSAUd7dtthciK5a44LSSYVQYBgHUnUcuGtQNl72ZcSFe4txLphmFtkymSE946jXXTSdCYoX7XtmFntXQ0AqVOpjqyeA/W+dJewtnKZDs0GOZgagzFRKSRrGDaPoImlXfbfO3gAgK9JceSEBiAPrMeQnQdsHspjsAhVDGWCgE9pjU+tVj7WNkM+Is3zb6S2LRQquhzhpAPMghjH6p7aCUs0AMf7V8Yzygt21nRcuY+ZJ18gKfNx9+UxwNu5lt3hHU5OOZWflVdYjZh1tDCkAD3+IB5/juoDsKy6Y21JICXQSw4whzGNeYBHnSTUipQcSwNtbNC33W7cysSWLBSwZmCty4DrHv07612VsrDvPTYgr2BEb4kr8IozdfDX7mc3Ikkw0jU+OnCPSjGE2LZIlWVvAg/KluY8CHtnCXWvMVLQcsa/orWU2X8ICxNeVKnge0q3ZeMKjIp94LKwTLAacCDMA+lQNsXv8RZlmIzcjm0kR3QPWhiXhz17afNxNxHxts3rl5bNksQGNsPcuEE5ipY9TszTqRwMSdlppOzJzbjRB2DdV7dqFLsBkZSxy8CNVGvDWimzLnRbSwKhVVizqQvO2w0kHhqPhXuO3DxODPSh1a0SJ1h0JJCzpBHuyeHW5AUY3a2cq4odNZIuhQyOQQeo3WHZ9ZfGT2VOxqZq9TdAssGgO8GAv3nygK1jIOrIDG5JggsNIlTM8jR+0kwBzqZf2WGtOpeGZGAYHRZBEg93Ga1qzBOik8FYi7l096YgzPDjER510wnsxx2JvXLlzLYti44DsNWCkhSiDUgwIJjQyJo/s/d/pE6cXI99XXLqroSrAa6jMpgxqIp03X2oz4WxcdpzWbZJ1IBZFMw2sSefnXX4dxetwblLTq2qt5/Qes1OkVNh8BN1UuC4uVglwx1hEBtOBPPs4UY2vuldQI5thgttbKuLg6zMQiyNJJYj6v1qZ97cMii9c0i9YugsYH5a2jMCFGnWthpn/JXjqaknHZXtKyAgkNGv1dZg8wYM91c1PLb638lPUb6FX71ezi/g0a+rpdsL7xByuk6DMp48hoT4CgdraBGHIABIGUHWY0jnBMAVbW/O0FxOFxFrOAAFde8KVkd+vzrbdXcuydndazkv3FfVl1UkEL3gRlPnW3Hcbq8ZOL1ctKg0ktNED2b2cLcwlgW1BxC3VVzEMFW50gJPZkUTHPSi+/2y7z3ke0pebUEaQArHkf1xS5uPZbD4i7YylWDKSOUgtOo01BXyirTxNwslq6iljmUFeeVjlbjyE5v2a5Ut0aYN7ZWijd4dj3yyNcBVR1QM0KI6zHTnLa0wezTB3UxA1LSro2sgAlShJI11Ugfrnso57QUuLYQNbBZsReMiToS/RrAHE2wCddCgqDuDfbLcCKTc6rAcAVGYECeYPHxFRTU6Ro2nC3zGvfXZ7Ph1tJBa5cVZ5CAzD4qB50K9n279+1fuC6YAtjNGssx6omOICkn9daZ9t41RYLmR0Rt3W0MgI6s08/dDVwwO1AlnMSBdvTcykGcziVUwTOVcq6fm1ptV2Y7nW0mYq7aVHm5wldJkE6QI+t2RrVeYbdb+8m51mIaYYjQ8ZY8ARx5mjWExbM7C8QmQDIBpEzmI1MMe2SfDWpX0u3wBED6qyeHNiug8OFPahJtYOmMxt1AIaAOfAHuAPWbxops3Fm4mZo4TmHukds0uvdTEXEUSQHQkRoQCDr26Cu+3N3n6Nvolw2mOptz+TuRrH6Hlp20SdIErYyE1qaWd1dtlwLV3RxprxDDip9D6UX2ljujA7SR5Cs4S3IqcXF0Dd+MJbuYVxc0K9ZD2MNB6zHnVXYBGz9VJOigDSXOiCe0mKs3eDDm4iiTDXrI0/8AcQ+mlSdlbOS5icwA6PDEgSPevsOs0n8xGygjm79lOWmmyo6jiqDmF2e3RoHcNcyjMYgFo1iOGtJvtQwt1cIWQlTbZXJXU5ODGB2SD4A07YzEBRQrE7UIQrAPeeR+01TgqIUmnZ8/YXbFwmBdd5BJB568zzps2LufiLloXiykszZFEdVNJJIGrFp0J0A8RXbD7m3r2IbIERM5ObkAT2Ace6rY2FsC3Yti3qTGpmPMLwHxPfWUYvdhGspXHmVJjth37USp14Rr8q47OZxetrwm4g9WFXHi8FkMzI+I8RQDa5tK1vMitcZwE01EcW8h8SKp+5CIrprWVteuAMRWVzmuSj9t7MVMoSIuEFDJjURBnxGtX3dVMPZt21A6NEVRERAGpEaa8aonYuJW8LFl9GS9bKntWRIP47Kc9pXXVxbDMuckKATDGCeA46A1stRxaixeWpJyRZ2yscmIt5MoZGGVs09YEax2j1oHtHBNYewrtLWro6Np1u4e5Fo5p+sjXLebthW5kDlZtXbdtTbjgNBM+Yyz6mge8zYq4LRdiFW6pPbB017tQf2a6EzBosSwZXQiQCNOAPCh+G2q+Y27hhFjMI4ydNeyRw7zUTA4oWMIHtr0pgMetBYkgGDB4H5VBs7SF69OXLoCVmesM2nDwpJoNrO+28YLF2/kM9Pb6RJPVF4FbMdwYm1pzObtoncxAs2cvEKgAgRoBA4nXhQ7bezhcFthE271rj+arLPx18QDXTeMxh7hHEK0R28vjVknPeK10mFKm6LTESpPANz8dCR50H2jte301pkuZmUjRWEFBBeRx4d/ZQfaOGvnIzFmJUlUZiwmGOgOgmBWu72A6UhroYXFgKvBSs6hhDFRpxHfWDUpZTN04ww0HPpLW9oJk6qPaMLOmWSZCnRScoMdmvOj2Nxl4Dq3Drw7jy9aXmPTYu85cHKqS1tQ3GBoJOXhEdmnbRX6QQssLZtHQHpreYgc+PvD4VcOpnqdPoLG3sbcZ7ZDPmIeSDBPuwNPAin7crajLg7ZuyGMyCIIgkQfQme/Wq32ps66cRcHTkjEqxsdYwiiFZuxCS3Lvpn3U2wlqx9FKG41gtbdy3vMGJJgg6a86JVBuTZai5RUUh02neXE4dxbjP79ueTocyT+0BPcTQ/Z+OsviLbW4VThi+Xgfyr2ypg8P8Nx5UG2M7s0W7gWSdAV6up06zAsfAR41rszCvhxiTmZmD5FLDKBaWXWT43GAHdVJ2ZNNYD+294LS2LpILBQVMD86FgE6fW7dKXcbtRMRaS4gKk+7OhUjloYkQdQa5rtRkw1xWRWMtJJ4gmZgaRBHbQV8QXKnJkE6RGSAvKB8vsrGc7i6NYQakrDe7Wzma2b11WYs50iSApK+WoOp0o/irQygcB+b0jT5AISf2Y8aj2GNm0iCA8ajrGSdTEePZUd8cwOVbZBbiYIY+Abrf176uCwiJu2yPfxa4UNcVczL7qTGa45CKCdT7zCTrAB4xS5itvbZsTfe9YuIBJtZQARxIByhh3HMfOjO8uGAw5DqYLAssgGBJgxwPPuiksmyR/hFSi5FhiRGvvCOsdTpxrLUm1KkbaWnGUbY8YhRdt29o2EZekRLlwQdAQCGkaSp0J5xPLXhtbbi4iEtsoaBnLHKEPAyTwE8D6V7i9j3LWHweGY9VVhxykKAgPbxJ8VmlboGDKw6rBte8rIIPnSb2u11CKUlT6FhY2+y2QT1jbe3EfXYMoX1J+NS8FZbD2lVQpYSWaOLsS7nzZmPnQDC7aTD2bCupbNqcuWFCnj1iJ15DspkubQkcAeOtbp4MOoPvY+4zDMFgHXUiB298dn9Kl4i1K6cBQnaWMygELMkiBy0OvPQdlEdlt+TMnkPLTv4caSkrotxdWFMCbaCZAHMzw8aiYzeNGc204D66sJbjoDwUacZ4UI2iLjSLaBjzggGDw48edDtkYMC/8AlSqnKSA7DjI7G8adioc8LiJWJHDloPL8a0jXbz3MeM4IyMVCnkFDfPj502XEIBK3VPPqMB98+c0p4R2fE9IxJYliSePAilIEHLtoEmva9Zqyufaa2UfvLtjDXcSt7B4UYRFiADqxB94r7qHlAp13T3jt3yi3QvSIQVnt7jykEjzrpgNjWbYE27fmoJ9Wk0m42+uFxl/JbDDKQgMhVL5GmBrpqI+Vaygp/UiGo4fQ+gARcTMFMdgOs944HzNK+3ejXQKARGuikkz2ATw4a8KVN0N+AFIvXcjqO8BwOyOJ7qHbc3/fpbgdFvIQMkwpGv1okHugD7auE28NZFOKXqi8DhgNtrbV7bsqgQ6liADOpWTzkE+dGMHhrZdroGpURHAkHn5kelVtuhjWx+IIfo0FsK2UAnOgbUanLxyjUcDVg2cVbsm6GAK2xnjU5VOp0TraQTA1gVzedHz/AC+tWaqP9OzXePDYjIi2GIMguVOViDOmbjE8Yj797qXDhSLhzMEBaOfDwnWoV3fPZrgt9JutIghUygj80Kylx4wfGoOM3mwty0Um5aQoCEDlWZNCCzHrBW/Z0Og5nsbSWTmim3gdsPsnCvDG9ndF/wAMOV1jgUXrE69nhUS7hUnImAzKRlIe2cpHOTdXXt1Op5GqO3kxme41wT1m6sk6DkAeMADSjG5b3GLDO8BZ946ajvqXDbG7HvuReKv0OXosDlWAIthBAHAQuvwqdY2hn06Jk/WyfY1VP0D/AOa/qfvrw4Mni7Hx/rUX2As7ErYDF3+jqYMl2UGO+htzenZC5lD4e4wksLSK54xxURz7ar7HYZbdp3cwsQT+sQo+JFVTgWcZshI6kPBjqllBnuzFaaV8wTo+iG3gw9tc0rbsllVAysxOYjTMDpry4Com2sbdsLimvqqWROVgvNmCW9U65kMJPI8Iqrd2942tjo7/AOUsnQ6SV745057zX8bi7bKuIstYuZJRlCEKCGBzAHNBE6QY5GpTcXU8Gkob/VD9CNsLalwbObFYiLtyGLSAM4QkAGBEkLxjnrNDNkbcsYu7ls2WswCzkxEEEcjE95HKue8KnCYJMHcILOWlkMqVLliNQCDw9aTLOfDN0iSyEEGDGh5GPnSjByjJrqy5TUZRT6IlWN4cbg2a1bxV0ZCVjNmQxoCFeVjyqw/ZZtzGY2+xvXUazbAzfkbQZ2acqgqgPIkxroO2qixl7O7MBEmYq6PZSluxhsP1gXuu1xgOIJUqog6+6B6mt0+5g12HnebC5rduyqBOluqhyAA5YZn1WYkKRP6VJ1rca4+J6M3LhtpDP1hIB91AZEMRrPIRpqKc94cW2awbaZrgZzbDaAOUKZj3AOTRPBKlpAskkcWbizH3mJ11Jk+dDimxKTRHxWzrbwHzqRlgk/mmR1iCD2HWSDSHt3ZBt4hlLAK8Op7ToDA9P3qctsbUdNUAZZAOvAEwSPuoPvaouYRjE5YaO7gdeWhnyqNSKkitOW12QsLcsZLK3LYOTNqwnWSRBHl8K63MSUVM3DKNfgakYHZ4NlGA0Kgzx46kzzJOtbNaVx0bIx7GUgFf3tD6inWBNq8CttnGurC8oOUDKYEx3+6T5yKDLvdiLd5FVSts6uco64j9IGB4fCi+1bGIsXFXRw7ZVYNMzyCydfQfOum0NnQdfU8++sow9TbR0TnUUkyfh95bNyAXCkdkBvVuVC94MTbbKyvBU8+Y8uffQ3EbGU8PgaiW9jvmAViPKfgKiUJ9xx1Ic6CGFtfSHVBd6MGTny5ssCQcpIkzA486aLQsqVW2LtxwNXaBPb1QoA+PjQ/Y2yegDM7h2bQFQQoXsE6z91Rdtbe+iAXcgfrZYLZTqCZGhk6cOyacIUqJ1J7nYzdJ3VlU3i9+MY7swusgJ0VAuUdwkE/Gva02me4ZhiLvdSpvTbbpekYQCo1Hdpr2cKsVLA7KNYzEnoBZtKlq3JLjNnd57TA+XdShKmJooVxrBEEHXxrlfpm352Z0V1bi+7cBn9ZePwI9DS4bRYMR9UZj4SB/uFbp2rM3gZPZdey7Qtj89Li/6c3+2m99uLh9q33umLeVU7dQtuI7OLUibgsw2hh8ozHM2njbcHmKnb0szYvEDQRcaewRp9lcXkbuJc+m2vmzZam3TX1Im9CILvSLlQOWORG4EcwPqgyDBiDIoTYugD3QTyJ1o5g9mEbNxl8pMvatK3ZluW2bTjqWUfs0BwliYnhIrrhJ8uxlOuYZ2ssWrenvO0HnCBB6HpP9PdR7dW50dp3BEs9i2B29JdUH4UM3kwTomGZhAawxUfrXM/rDLxAo7uDgb13J1R0SXxcZp4lbbgKBxMMyN2a1GonKHz9mEaUvgY8XdRCoYgFyQo5sQCxjyHGg27u+GHxF0Wsr22bRM0Qx7JB0PjQr2r23tYq0wJC9CQn6xLq/+kr8Kr+1cKsGXRlIZT2EGR8aFEC3faawTBqJjNeTziT9k+VKXs+wQu469aYAq1m+rjuLKNPOKdd/dmPjtnWr1hSzDJeCgSWVkIYAcyMwMfomkbY2022fjL1w29RlDq3vBbkXHUdjchPZrT5IFkgLhWtXrll/eRip745+Yg+dFNnYe61i8y3GAsspVTqo4TGuhHHh4V234sj6UuKQ5rWIVWVu8KB8gD61J2JiB9FxS6FcwY666KDMdnUo1nKWmtvO198j0/TP4FNbuJxbBBLm2rMFkDKojMZY9w0nwrzB4+NDBB4g/jWhvSGWietxA5gmYMcdQNO4VO2rsa/hktvet5BdBKg+9pE5h9XiNDrVRe0Us8zVcPba/bVfcd007ASAR+O2rd2Dg0badpU0KW2ZgNBEKoPqwGnGqw3Vw9vpBevaqkMqfnt+Uy/shk9SKcN1d6ktY9b+Kv3VtKH0CkhuSKVTiBObTmoqXBylu6ItTUYNdWXVauWjckalFKjRoglZg8Drl9K2xgBHuz5A1XO9ftCw9q3ZbZxt3Gdy10EOMqgaKQQCss09nVNddgb/AF+9BfCKq69dbx8JCshJE6amO+a1bSVsyhFyx1CW91preExFy3Ai056pIGg4lT+BS5hNvYm7YKEKqlIZguhleRaZ0PKjm8e8c4e7lVc2UyrNAYRqJjQkc9ar2/vKl22tjDjIuQcT7ogaD84iuScnL+w64QUP8gsHa+IsXH6C/ct9dtEcheJ+rMH0olgd/Nog/wDxJPLrW7ZPxSgOOt5bjDv+yuGFPWrqXucrG5d98c122Xv5gHU5clsAwf0UBqxtk7Ws3xnUrnPFG4qe6NSPKqTQw6/rD50Tt3SHiAQTpPI0pYY48i7OhU++FHjbuD4stRruzOtwBXsSPjz9Kr7Cb0YmyIt3rigcs2YejyK8O9mNugh8U+UzoFtrM8pRQ1TY6H3aV5ZhTOUAGOAI4j5cdanbu4DCXlP0nDpe62mcBgNI91tO3Wk7ZCZbCaRmkx4nT4RXSxjbyYrDiyTLuFK8mWevI7hrPLWspSrJrGO7BaC7v7M/9Fhf/wAe3/xr2thWUbiKEe3aWO2uOJt1KwokV3+jLxImPT+dSotlsr72jECxZB4m4SPBVIP8S0I3I2b0q40hQzLhXVQdeu4OWO/qH1o77WVkYd+QNxfUKR/Ca6+ySBaxDczcUeQWR/EfWuhKomTyxL3FvZdoYUjncC/vAr9tE9v3F+l4ogz+Vfz1PyPyofj/AO7bTcqBFvEZ1HAQWzAeQb4UPRiSFnrOQvD84xPedaa7kvsWDtCxk3eA/O6O5/5Lyv8AJhSHg390ceJ+FWH7QyLez1spOUNatjwQEie33RVd7KwzO2VRr29k6cRw40aYTHPf7FK3QW1YN0doSo1ZcwXie8AGO7vr3cnbf0QPda2z2Y6xUiVjWcpInTwrzenZ9lENzUNIGg9eEdlCfpCnBXLNskl3tqsiBLsNNCYHUPHtpYcaHT3WWZvHsmxtjBBsPcUspLWn7G+sjjisiAew5TrGtGYzAvauNavKbdxTBVhqD9x4gjQ+FfQHs42T0GGgTlYiJ5wILeZn0o7tTYeGxMdPYt3cvAuoJHcDxA7qlMBa3Uxi2Nj2L14wtuzmJ7gTljtJEAeIqn9vX2vvfvkf4r2GAHINbYgd8CB5VbvtTwYbB27Cno0NxRCgAQikqsR7oIBgR7opM3Z2EMRhsjsbboVGdYzA22uqOP6JGvhQwiKWNxLHB4e2ykdG7QT+k12QR2iARI51L2Tc/uWNicwFoceTsVmPMz5VM31GHtWxh7OZnW5nuOTJLQVOYxBOvAQBFR9wVW7cvYdwct6yRpyykQf9R9KqLuI5Kmeez3Djp7l0gHIoCyODMZkdhAU6/pVO9peIZ0szwDsPMgR8jXDYi3LFx0aI4E/pKY4cufwqTvFhzft5QdQcw7+OnxrPO6x4oWunAVVn3VA9PDzNRr+I5T5HWtb6FTDCCOXMVyawYZwNFgE9paQAPEZj+ya6HLBkkdsNmJAJnz+0Ubbat2wmVHYDjAJjlOoIKzA1BBoFBUJpGkg9oJkT8vSuyOrsBcYqvM/z5eNFrbkeU7R0x21bt8EOYtqJyqIHdJJJbzJqXtPZowr21jrrk6TXg7KCwHZE5T3zUzYuHtPi8PbTKbZe2DqDMsc0nmda4by4vpOluni1zpP3nzfbWVJci223kgbatDNmB1bl4UOsaHyoptlAHE9hHoagWF1PcpPprVohmPchgY4EGieI0aew0Jcc6PbQtcD2gGlIqJ6W0+6t8BhDduJbPB2A8ATr8Kg4XVe8GKObqa4m34k+gOtZstDVte7k0HLQUYXE4XZ4+kYh+u4CLAJbKNSFUa6k5ie0gToKA7ZM3UXtdR6kClf2lXLj4y4xMrbCoo/NAEnTvJJ/pWeLVlrdTotK17SdmkAm+VPYbbyPRSKyvntiaytNqM9xfi40AQJArQ44f1oXtF8txgO2ofSmaqh2cPaAwuYMmNUdGHrkPweoXsxxeWzeWf8AuAx4qPurbeK10mHdQSD73jl1jzpY3dxosOdYVxDd0TB8tfXuo6C6nXebZh+kX7uaRmFwSDrnksAYgZTpHYRQXZtstetqObp/EKfNp2Js3CSvuMZPDQHspU3XuEYlCQAFDE6d0c+B1prkS+Y179Xpwup+upHedRp6k+RpJ2VdyXFf81lPowP2U4byKb9khVMghh4jSPSaUMCwS4C6TB6wI1HeBQuVD62PWKTplyusg8qmbs7jW75ZWuOttXtuVHEgC6Mobis5uOp4x2jphcpUEaggEGmvcojNcHOF+2k6DIS2rt23hotqslQNBoqiNAT4UubM9rGDe8bVwNaEwt09a0eWpGqieZERqSKqzfHbt+5icTbZ4QX7q5QAJAdgATxOgiluihH1JvBs4YiwVEFtHQ8pAMa94JE99JewsNkR5BUtdfQ6HqpaU/GaNey43P8ApeG6SZhss8cmdsnlliO6Kib83wUUqwJVypg6gwTGnA6UUJCDtrdF0S4UZXGUkDXOY1iIIJ89aHezm2RjF4e64YEHMNOPCJmBHeaO3MfdjRj+PGl3Yrf3kOfrO0+YJpVtwVmWRm2zs8tiGKe6Yk9/OuR2aRzNE845RXmKvgKSeABM+Ap0SV3vDaKX3kT7v8K0PLEjJMKWBM9okTw5Bm9aI47pMQ4ME3ZCBVBltYAAGskk+lRcfs29ZudFetvbuCOo4IbXhE8QTzHGrTxQEfEkZjlMDhU3Z2HWAza6yJ4ad3OoLoc2WIMwZ5ePZU61MCk2CCOzby/TLbQBDKRH52mtcb1rpctvMSbjog5mWYDt7ah27uS8jdhB9D/Sp+H/ACd9XGotkuO9h/hD97KfBTUjOW8kFyw4Z3jwJJHyoTZbU/qt/CaI7Xt5VCknt7e7jQxDrHcfkapchGyDMQo5kD10py21hB0RaNViPMgRShs66q3UZvdDKT4A0/7RWbT/AKs+mtKfMaE7BGSQe6NeetG9kqUvW2ZCIYGfPnQO4skeNTsNZUECJ4a9/wCO6oZaG7a5JxFpRxN23/EK33x3WxLYi5dt22u27pUgIMzKcoDAgaxIkHh1u6iWycGfpS3G7GjuJjh5TT/hqylG1RpCbi7KMPs7xrajC3ADyzIPgWkVlX+DWUU+496/5RWe8lmL2nMUPFmjGLJdszESeX45Vz6Aju/HbXQYIgHDjhUC7u7YZixtgknkWE+QMUfFsc/617k7o76AFzeTCt9HZbaE+6IAkhR2D0obujsVwzPdQqNAuYRPbodYp3/AFahJOv8AKgRzawO4dgFQsRsqy5l7SM3aVB/rRYLz+NcnWf5fjupUBGLaQOHZ/KiO7N7LiU/TDA+hI+IFRBY4R/SjG7ezrbXVuMTmtZiiz1ZZchYiJJAzAfrnugYxM9oPs8xD4x7+HTPbvHOwXVkc+91eJBPWnvM9++7XsjZmV8WxS2DJtgjO/cSJyA9sz4cauEVuKViK+9oe3mt2jhMKwtsFCsyj/DSICJHumI15DhrqK83TtXVa5bZiydGGGpIGV1UQDw0c1YW1fZy1y47jECHZm6ymRmJJ4HXj3VzTYFnCo1tG6S45GdzwAXgqiTGpnn8gGIEnDAzQnBbslLgbpJVSSBl6x0MTTPbsiupsRw4/1oHZA6Cl/fF2SxofeYA+EH7abmsH4TWuIws6aedMRr7ONs7JwmHVylxMQUHSXLqEmeeQgEIkzERx1o5szaeH2xjbbJZVkwp6UXWGsnRQPEiY5ZAdCBSLtPYT3JhgPKgdvZ+Lwjl7F1kY6EpIkdhHA+dTQyw/bhs7Dolm6ttFvtcyBgIYrBLTHGB291VWEh1GpEj48u+id1MXiWFzEO9wroC3IHjAEAT3VCweHd8WoAJCMZ7AB9+lMDTaWCBEANmB0gTPdUrZGw7vv3CdAcqkzB7TTnawXDSPKuv0Tl8u+gBAO7l99XPWJ1MT8c0x5VIXdtUXNJLRx5elOwtcu6tHw8iImfhTAqzDJ1kJAIzDjqOIp0xiN0LmW4a8/wACgWOWxZvNCsxVuDOgWeMiVJqHtTbNx4ObL2IhaB4k+94cKTyBvZSbkEzAJ4fdRnYlnPeT82Zn9XX7KXdn4uSWbjGn3zypl3ZQNdLENmUaSxjXSk0UPdu5rP4/lRzA7WIHW1+B/nSyg51KV9J4GlSBDYu2EI91/KPvr2lhZIn76ylQ7PMgUaCuQ1Ov476wsT4VsqAiDp5fbWpB7bWO7x/nWNqeNekc+PZNbmO80hni6c4nzNZEcP5VsvGPWftre4/nxjT08PCgDi7Dh8q1tKZJ5d9dhb17xzroojt+Oumvx50xHDo67YUENmBKkdnGvVMT28h3V7a4a8z+PGlQBddruo1APlr8Ky5tt+OUQeHH76ErMaan4V4I5zy/HjRQEvF7Rd5BaBwgacu6hVy3r511uGO7UceP40rRyAAPx606EclT7a3C8fCeNegdvGukwOelFAR3JEV1EEfbXl3sOprpbXTlE0AcYgfjurvaw4PETr+Px31zJ9PtrsoAHedNedMDlcw47OZrX6KoPDjp+IqQx8/wawiB4ye+pGaLaGg4COFcms+VSWIAC8p/H47q8K68NZ/n6UxEEWp4DtrqU08fjXRxwEjs/EV0yDw0499AwTi9m2nEsit4qDx8qW9v4fC4ZZFhDcb3Vyj1PYKdwI0j7++oG09gWb8dIkxw1IPH9E8KQFTm0MhMQZ48OOunIAdnfTNuXsm6C1wiFZQFnnqDIHZofWmrB7qYVGnoVMajMS38RIo0toQI05d2k1LGkDbFgz39n451KVSOetSSvb8te6tXtTwPwpUUc1Y9h9B99eVJCdsfjzrypoLICnreVZb5+H3VlZWxB3PD0ry4et+O+vaygD3n+O6sxHveR+2srKBnkxw/O+2ulrifE/KsrKBHRPx8K8Xj5n5V7WUAaYQfP766MPmPnXlZSA4NxHn8xWlwaCsrKoR0Th+Oyt24etZWUAcZ6w/HI1ItDX96srKAOGF5/rH5tWwFZWUAdcWIiK6KND4/bWVlIZyucB+qPlXPEHVfP5VlZTEep73kPkK6NxYcs3CsrKGBmGPUrY+6PH7aysqRmlw6r4/dXS6dPJfmKyspFG1w6r4H5muZ4+VZWUgOiHSsrKygD//Z"
#]
#for img in imgs:
#    run_visualization(img)